<a href="https://colab.research.google.com/github/21-Vaibhav/Major-project---24/blob/main/Prediction_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Step 1: Load the dataset
ds = load_dataset("L-NLProc/Realistic_LJP_BertSum")

# Step 2: Preprocess the data
tokenizer = AutoTokenizer.from_pretrained("law-ai/InLegalBERT")

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True)

encoded_ds = ds.map(preprocess_function, batched=True)

# Step 3: Manually split the dataset using the datasets library's train_test_split method
train_test_split_ratio = 0.1  # 10% for validation
encoded_ds = encoded_ds['train'].train_test_split(test_size=train_test_split_ratio)

train_dataset = encoded_ds['train'].shuffle(seed=42).select(range(10000))  # Increase subset size for better accuracy
eval_dataset = encoded_ds['test'].shuffle(seed=42).select(range(2000))  # Increase subset size for better evaluation

# Step 4: Fine-tune the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained("law-ai/InLegalBERT", num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,  # Increase the number of epochs
    weight_decay=0.01,
    fp16=True,  # Use mixed precision training
    lr_scheduler_type='linear',  # Use a learning rate scheduler
    warmup_steps=500,  # Number of warmup steps for learning rate scheduler
)

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='binary')
    acc = accuracy_score(p.label_ids, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

# Step 5: Evaluate the model
eval_results = trainer.evaluate()

print(f"Evaluation Results: {eval_results}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at law-ai/InLegalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.689400,0.675967,0.589500,0.035253,0.600000,0.018160
2,0.679300,0.667701,0.600000,0.297012,0.541667,0.204600
3,0.669000,0.674499,0.597500,0.392453,0.521042,0.314770
4,0.611900,0.690151,0.580000,0.490291,0.491484,0.489104
5,0.540300,0.741161,0.588500,0.480757,0.501976,0.461259


Evaluation Results: {'eval_loss': 0.7411605715751648, 'eval_accuracy': 0.5885, 'eval_f1': 0.4807570977917981, 'eval_precision': 0.5019762845849802, 'eval_recall': 0.4612590799031477, 'eval_runtime': 15.9847, 'eval_samples_per_second': 125.119, 'eval_steps_per_second': 7.82, 'epoch': 5.0}


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
# Define the path to save the model and tokenizer in Google Drive
model_save_path = '/content/gdrive/MyDrive/saved_model'

# Save the model and tokenizer
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

('/content/gdrive/MyDrive/saved_model/tokenizer_config.json',
 '/content/gdrive/MyDrive/saved_model/special_tokens_map.json',
 '/content/gdrive/MyDrive/saved_model/vocab.txt',
 '/content/gdrive/MyDrive/saved_model/added_tokens.json',
 '/content/gdrive/MyDrive/saved_model/tokenizer.json')

In [15]:
model_load_path = '/content/gdrive/MyDrive/saved_model'
tokenizer = AutoTokenizer.from_pretrained(model_load_path)
model = AutoModelForSequenceClassification.from_pretrained(model_load_path)

def predict(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

    # Make predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted label
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=-1).item()

    # Map the predicted label to the corresponding class name
    label_map = {0: 'Against the appellant', 1: 'In favor of the appellant'}  # Adjust this mapping as per your dataset
    predicted_label = label_map[predicted_class_id]

    return predicted_label

# Example usage
text_input = "C Gupta J This appeal by special leave is directed against a Judgment of the Calcutta High Court setting aside in revision the finding of the trial companyrt on the issue whether the relationship of landlord and tenant subsisted between the parties in a suit for ejectment The issue which arises on the interaction of two statutes the West Bengal Premises Rent Control Temporary Provisions Act 1950 and the West Bengal Premises Tenancy Act 1956 which repeals the earlier Act but keeps it alive for proceedings pending on the date of repeal involves the question is the right companyferred on the sub tenant by the 1956 Act of being declared a tenant directly under the superior landlord available to a sub tenant against whom a suit for ejectment was pending when that Act came into force  The appeal turns on the answer to this question The material facts leading to the impugned order are these The respondent was a tenant of premises No 171E Gopal Nagar Road Alipore Calcutta and his landlord was one Jagabandhu Saha the owner of the house Dilip Narayan Roy Chowdhury was a sub tenant under the respondent in respect of the ground floor flat paying a monthly rent of Rs Sub tenant Roy Choudhury served a numberice under Section 16 2 of the 1956 Act upon the superior landlord and applied under Section 16 3 for being declared a tenant directly under him On July 31 1956 the Rent Controller recorded a finding on this application that Roy Choudhury was entitled to the declaration asked for overruling the objections raised by the respondent On February 23 1957 the Rent Controller companycluded the proceeding under Section 16 3 by finally declaring that the sub tenant was a tenant directly under the superior landlord with effect from that date and fixing the rent payable by him The appeal preferred by the respondent from this order was dismissed by the appellate authority In the meantime on August 21 1956 the respondent had made an application under Section 14 4 of the 1950 Act in the suit for eviction which was pending Section 14 4 of the 1950 Act permitted the landlord to make an application in the suit for an order on the tenant to deposit month by month the rent at the rate at which it was last paid and also the arrears of rent if any and provided that on failure to deposit the arrears of rent or the rent for any month within the period prescribed for such deposits the companyrt would make an order striking out the tenants defence against ejectment so that the tenant would be in he same position as if he had number defended the claim to ejectment On this application the Munsif on September 26 1956 directed the appellant to deposit a certain sum as arrears of rent and also rent month by month at the rate of Rs After the declaration of tenancy under Section 16 3  Roy Choudhury was permitted to amend his written statement in the suit by adding a paragraph questioning the relationship of landlord and tenant between the respondent and himself It is unnecessary to refer to the various proceedings in the suit that followed in the companyrse of which the High Court was moved more than once by either party On January 24 1965 Roy Choudhury died and the present appellants were substituted in his place in the suit as his heirs and legal representatives On November 1 1965 the Munsif framed an additional issue being issue No 9 which was as follows Has the alleged relationship of landlord and tenant between the parties been determined by final orders dated 31 7 56 and 23 2 57 passed by the R C Rent Controller Calcutta in Case No 243B of 1956  The Munsif took up for companysideration the application under Section 14 4 and the additional issue No 9 together and by his order dated February 20 1967 found that the Rent Controller had jurisdiction to pass the order under Section 16 3 declaring the defendant to be a direct tenant under the superior landlord and that the relationship of landlord and tenant between the parties ceased by virtue of the order made under Section 16 3  The additional issue No The plaintiff moved the High Court in revision against this order The propriety of this order is challenged by the tenant defendants"
predicted_label = predict(text_input)
print(f"Predicted Label: {predicted_label}")

#c

Predicted Label: In favor of the appellant


In [8]:

model_load_path = '/content/gdrive/MyDrive/saved_model'
tokenizer = AutoTokenizer.from_pretrained(model_load_path)
model = AutoModelForSequenceClassification.from_pretrained(model_load_path)

def predict(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

    # Make predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted label
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=-1).item()

    # Map the predicted label to the corresponding class name
    label_map = {0: 'In favor of appellant', 1: 'Against the appellant'}  # Adjust this mapping as per your dataset
    predicted_label = label_map[predicted_class_id]

    return predicted_label

# Example usage
text_input = "CRIMINAL APPELLATE JURISDICTION Criminal Appeals Nos 41 and 77 of 1973 Appeals under Section 19 of the Contempt of Courts Act 1971 from the Judgment and Order dated the 5th February 1973 of the Orissa High Court at Cuttack in Criminal Miscellaneous Case No 8 of 1972 K Ser G L Mukhoty and C S S Rao for the appellant in Cr A 4173  Rath and B Parthasarathy for the appellant In Cr A 7773  S Nariman Additional Solicitor General B M Patnaik and Vinoo Bhagat for respondent No 1 in Cr A 4173 and respondent NO 2 in Or A 7773  Rath and U P Singhfor respondent No 2 in Cr A 4173  K Sen and C S S Rao for respondent No 1 in Cr A 7773  The judgment of the Court were delivered by PALEKAR J This is Criminal Appeal No 41 of 1973 an appeal by one Baradakanta Mishra from his companyviction and sentence under the Contempt of Courts Act 1971 by a Full Bench of fiVe of the Orissa High Court The Judgment is reported in ILR 1913 Cuttack 134 Registrar of the Orissa High Court v Baradakanta Mishra and Ors The appellant started his career as a Munsif in 1947 His career as a Judicial Officer was far from satisfactory In 1956 he was promoted on trial basis to the rank of a sub Judge with the observation 28 7 that if he was found incompetent suitable action would be taken In due companyrse he was companyfirmed as a Subordinate Judge As his work was for unsatisfactory he wag reverted to his substantive post of a Subordinate Judge on January 4 1963 The order of reversion was challenged by him in a Writ Petition which was dismissible by a Bench of CJ and  J The case is reported in ILR 1966 Cutback 503 An appeal to the Supreme Court was dismissed on February 6he 1967 While working as a Subordinate Judge after reversion was suspended from service from 15th May 1964 to 9th April 1967 during the pendency of a disciplinary proceeding against him that proceeding ended in a light punishment of two of his increments being stopped From the above order of punishment the appellant filed on 10 10 1967 an appeal to the State Government The State Government by its order dated 15 7 1970 allowed tie appeal on the ground that the Public Service Commission had number heed companysulted by the High Court before imposing the punishment and that the Charge Sheet served on the appellant having indicated the proposed punishment vitiated the disciplinary proceedings After the case Was sent back to the High Court the charges which had been earlier established were framed again and served on him on 13 2 1971 and we are informed that the proceeding is still pending In the meantime it appears he was promoted to the post of the Additional District Maggistrate in February 1968 though the High Court was of opinion that he was unbalanced quarrelsome reflect and undisciplined The High Court specifically observed that though the appellant suffered from these defects It was sincere and working and the other officers who had superseded him as Additional Districting Magistratres were number much better The promotion was made on trial basis for a period of one year with the observation that if during that period his work was found to be unsatisfactory he would be reverted to the rank of Sub Judge In that year the High Court had to face an abnormal situation by the retirement of many District Judges on account of the decision of the Government reducing the age of retirement from 58 to 55 years Many vacancies occurred and the appellant was then promoted as an Additional District and Sessions Judge on trial basis for six months in July 1968"
predicted_label = predict(text_input)
print(f"Predicted Label: {predicted_label}")

# Load a pre-trained LLM (e.g., GPT-2)
from transformers import pipeline
llm = pipeline('text-generation', model='gpt2')

def justify_prediction(text, predicted_label):
    # Generate a justification using the LLM
    prompt = f"The legal text is: '{text}'. Based on this text, the predicted outcome is '{predicted_label}'. Justify this prediction."
    justification = llm(prompt, max_length=1024, num_return_sequences=1)[0]['generated_text']

    return justification

# Example usage
justification = justify_prediction(text_input, predicted_label)
print(f"Justification: {justification}")

Predicted Label: Against the appellant


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Justification: The legal text is: 'CRIMINAL APPELLATE JURISDICTION Criminal Appeals Nos 41 and 77 of 1973 Appeals under Section 19 of the Contempt of Courts Act 1971 from the Judgment and Order dated the 5th February 1973 of the Orissa High Court at Cuttack in Criminal Miscellaneous Case No 8 of 1972 K Ser G L Mukhoty and C S S Rao for the appellant in Cr A 4173  Rath and B Parthasarathy for the appellant In Cr A 7773  S Nariman Additional Solicitor General B M Patnaik and Vinoo Bhagat for respondent No 1 in Cr A 4173 and respondent NO 2 in Or A 7773  Rath and U P Singhfor respondent No 2 in Cr A 4173  K Sen and C S S Rao for respondent No 1 in Cr A 7773  The judgment of the Court were delivered by PALEKAR J This is Criminal Appeal No 41 of 1973 an appeal by one Baradakanta Mishra from his companyviction and sentence under the Contempt of Courts Act 1971 by a Full Bench of fiVe of the Orissa High Court The Judgment is reported in ILR 1913 Cuttack 134 Registrar of the Orissa High Court 

In [12]:

model_load_path = '/content/gdrive/MyDrive/saved_model'
tokenizer = AutoTokenizer.from_pretrained(model_load_path)
model = AutoModelForSequenceClassification.from_pretrained(model_load_path)

def predict(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

    # Make predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted label
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=-1).item()

    # Map the predicted label to the corresponding class name
    label_map = {0: 'Favor of the appellant', 1: 'Against the appellant'}  # Adjust this mapping as per your dataset
    predicted_label = label_map[predicted_class_id]

    return predicted_label

# Example usage
text_input = "the instant revision petition is directed against the order dated 27082018 passed by the Additional Session Judge Special JudgeWest District Tis Hazari Court Delhi in Criminal Appeal being CA No 153 2018 . the respondent handed over three cheques drawn on Bank of Punjab Limited Rajouri Garden Delhi bearing number 327226 dated 13042003 for an amount of Rs 3 lakhs . a complaint was filed under Section 138 of the NI Act before the court of the Metropolitan Magistrate west district Tishazari vs. the petitioner is a leasing and financing company The respondent financed a bus with registration number DL1 PA 5798 and at the time of taking the loan 36 blank signed cheques were handed as unpaid dishonored reason Funds c Notice as required under the CC No8073 2016 . the petitioner herein took a loan for purchasing a bus with registration number DL1 PA 5798 and at the time of taking the loan 36 blank signed cheques were given as security towards CRLREVP 774 2018 the repayment of the loan It was stated before the Metropolitan Magistrate that as the vehicle is in the possession of the respondent herein and the cheque given by the accused had been misused It was also stated the No Objection Certificate dated 30082003 along with form 35 was issued by the issuing authority . the . averments of the accused petitioner . herein he took the loan for getting the vehicle converted to CNG and entered into a new lease agreement but neither was the said vehicle . it was stated that in October 2002, the re-election of the vehicle to be converted to a . the accused petitioner herein and the complainant respondent herein had dealings in relation to two buses It was found that the petitioner had another bus bearing No DL1 P 7279 which was under the loan of the Motor General Finance Ltd Company in the year 1995 The Metropolitan Magistrate held that the claim was consistent with the testimony of the accused wherein he has admitted that he cleared his loan with MGF India Limited in 2001 CW 1 also deposed that one bus was in the name of the accuser and the other bus financed and the amount of Rs 8 Lakhs was paid for discharge of the NI Act . the loan was taken over by the respondent company . a sum of Rs 8. Lakh was lent to the accused in the . j Soni learned counsel for the petitioner contends that the failure on part of the respondent to submit their books of accounts is fatal to the case of a respondent . he would contend that the courts below ought to have drawn adverse inference against the complainant . the court in MS Narayana Menon v State of Kerala reported as 6 SCC 39 to contend that it was for the complainants respondent herein to produce the books of account . it cannot be said that there was a debt due and subsisting and the cheques have been issued for discharge of liability for the second bus bearing registration No DL1 P 7279 stood in the name of the accused . The respondent deliberately did not produce the book of accounts despite the fact that the i.e. in the present case other than mere ipse dixit of the . the law law of commercial world legislated to facilitate the activities in trade and commerce making provision of giving sanctity to the instruments of credit which could be deemed to be convertible into money and easily passable from one hand to CRLREVP 774 2018 another To achieve the objectives of the Act the legislature has in its wisdom thought it proper to make such provisions in the Act for conferring such privileges to the mercantile instruments contemplated under it and provide special penalties and procedure in case the obligations under the instruments are not discharged The first bus bearing registration No DL1 P 7279 was given in discharge of debt by a specific object of making a special provision by incorporating a strict liability so far as the cheque a negotiable in the present day world are likely to be a smooth and healthy manner ultimately affecting the commercial and . the . petitioner only seeks to take advantage of the fact that the respondent did not produce the books of accounts to rebut the initial presumption which was for the petitioner to show that the amount of loan taken by him and the amount that should be repaid in order to discharge the initial burden . the scope of the revision petition under Sections 397 401 CrPC read with Section 482 CrPC is extremely narrow In State of Haryana v Rajmal reported as 13 SCC 670 the Supreme Court observed as under the jurisdiction of reappreciate the evidence and come to its own conclusion on the same when it was without any reference to the evidence on record or there is a glaring defect in the procedure or there was a manifest error on a point of law resulting in a flagrant miscarriage of justice report in state of Kerala vs. in the case in hand the High Court has exceeded its revisional jurisdiction in interfering with the conviction of the respondent by reappreciating the oral evidence . but the said revisional power cannot be equated with the power of an appellate court nor can it be treated even as a second appellate jurisdiction Ordinarily therefore it would not be appropriate for the High court to re appreciate the evidence and come to its own conclusion on the same when the evidence has already been appreciated by the Magistrate as well as the Sessions Judge in state of Kerala v Puttumana Illath Jathavedan Namboodiri resulting in a flagrant miscarriage of justice . the high court has not been able to prove that the judgment of the high Court is a supervisory jurisdiction exercised by the Supreme Court for re-examination of any finding sentence or order unless any glaring feature is brought to the notice of the court . learned counsel for the petitioner has not been able to demonstrate that the findings of the courts below are perverse . the fact that the respondent did not file the books of accounts is not fatal to the case of the respondingent . it was open to the petitioners to produce his books of account to rebut the presumption and bring out a prima facie case that there was no debt due and payable on the date the cheques were dishonoured"

predicted_label = predict(text_input)
print(f"Predicted Label: {predicted_label}")

Predicted Label: Against the appellant


In [17]:

model_load_path = '/content/gdrive/MyDrive/saved_model'
tokenizer = AutoTokenizer.from_pretrained(model_load_path)
model = AutoModelForSequenceClassification.from_pretrained(model_load_path)

def predict(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

    # Make predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted label
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=-1).item()

    # Map the predicted label to the corresponding class name
    label_map = {0: 'In favor of the appellant', 1: 'Against the appellant'}  # Adjust this mapping as per your dataset
    predicted_label = label_map[predicted_class_id]

    return predicted_label

# Example usage
text_input = "In these appeals relating to the assessment years 1957 58 to 1974 75 the main question which is involved for companysideration is whether the appellant Trust being a public charitable trust is eligible for exemption from income tax On 28th November 1941 a partnership deed was executed between K Rajgopal and VS Nanjappa Chettiar By this document the said parties agreed to carry on business in partnership but clauses 4 to 8 of the Deed companytained an agreement amongst the partners that out of the net profits of the business after payment of all charges and expenses 80 per cent shall be set apart and allotted to charitable and religious objects The amounts which were to be credited to the Charity Account in the books of business were to be utilised by the trustee for carrying out the objects of the Trust The partnership deed appointed one TN Venkatarama Chettiar son of one of the partners as the sole trustee The power which was given to the said sole trustee was companytained in para 8 of the Deed which reads as follows The said TN Venkatarama Chettiar shall from time to time after acceptance of the trust operate on the fund in execution of the trust and spend out of the said fund in companynection with temple festivals in Madras and other places like Conjeevaram Tirupathi Srirangam Salem and other places medical relief the giving of alms including food to the poor on occasions of Hindu festivals as selected by him in his discretion of the gift of sums of money to poor deserving persons for celebration of marriage and generally on any object of Chcultries work houses hospitals etc By a subsequent document executed on 26th August 1943 this clause companytaining the power of revocation was deleted On 1st July 1944 the sole trustee executed a document purporting to be a declaration of trust It referred to the carrying on of the partnership business by the earlier founders of the trust and for establishment of the trust for charitable and religious objects and then proceeded to make a declaration of trust specifying a number of objects most of which were different from the ones companytained in the partnership deed of 28th November 1941 One of the clauses in this deed of 1st July 1944 was regarded by the Income Tax Department as number being charitable in nature It appears that one M s East India Industries Madras Pvt Ltd a donor to the appellant Trust in respect of the donation given to the Trust claimed exemption from tax under section 158 of the Income Tax Act 1922 While companysidering the case relating to this donation the Income Tax Officer in East India Industries case came to the companyclusion that by virtue of this clause relating to number charitable object the Trust created by the Deed dated 1st July 1944 companyld number be regarded as a Trust having charitable object and  therefore number entitled to exemption under the Income Tax Act This matter was carried to this Court and in the judgment reported as East India Industries Ltd Vs Commissioner of Income Tax 65 ITR 611 the view of the Department was affirmed and it was held that the document dated 1st July 1944 did number establish a public charitable trust because one of the objects was number charitable in nature and there was numberhing in the Trust Deed which prevented the trustees from applying the whole or any part of the Trust property for that purpose In the assessment for the years 1957 58 to 1974 75 of the appellant the question arose whether it companyld be regarded as a public charitable trust entitled to the benefits under the Income Tax Act The Income Tax Officer as well as the Appellate Commissioner came to the companyclusion that the appellant was number a public charitable trust Before the Income Tax Appellate Tribunal it was companytended on behalf of the appellant Trust that the Trust had companye into existence by virtue of Deed dated 28th November 1941 and the Declaration of Trust under a deed dated 1st July 1944 was invalid By a detailed order the Income Tax Tribunal came to the companyclusion that the question as to whether the Deed dated 1st July 1944 was validly executed was number in issue in East India Industries case It further held that since an irrevocable trust had been created by virtue of partnership deed dated 28th November 1941 neither the founders number the trustees had any right in law to vary the objects of that Trust by executing another document dated 1st July 1944 The Tribunal therefore companycluded that the decision of this Court in East India Industries case companyld number prevent it from examining the terms of the objects of the Trust companytained in partnership deed dated 28th November 1941 inasmuch as that a deed dated 1st July 1944 being regarded as void companytinued to exist After examining the objects of the Trust as companytained in the Deed dated 28th November 1941 the Tribunal came to the companyclusion that the objects of the Trust were charitable in nature and therefore the appellant was entitled to be regarded as public charitable Trust which would enable it to get the benefits under the Act At the instance of the Revenue in respect of the assessment years 1957 58 to 1961 62 the Tribunal referred the following five questions of law to the High Court Whether on the facts and in the circumstances of the case the Appellate Tribunal was right in holding that the assessee was a trust companystituted by the deed dated November 28 1941 and number by the deed dated July 1 1944 Whether on the facts and in the circumstances of the case the Appellant Tribunal was right in holding that the question whether the assessee was a trust whose objects are wholly charitable and religious has to be determined solely with reference to the trust deed dated November 28 1941 and number with reference to the trust deed dated July 1 1944 Whether on the facts and in the circumstances of the case the Appellate Tribunal was right in holding that the trust created by the partnership deed dated November 28 1941 companytinued even after the dissolution of the partnership and the assesses trust was the trust companystituted by the deed dated November 28 1941 Whether on the facts and in the circumstances of the case the Appellate Tribunal was right in holding that giving cash grants for the needy and deserving persons to meet marriage expenses is a charitable object and the assessee was entitled to exemption under section 4 3 l of the Indian Income Tax Act 1922 Whether on the facts and in the circumstances of the case the Appellate Tribunal was right in holding that the assesse trust was entitled to the exemption under section 4 3 1 of the Indian Income tax Act 1922 in respect of the income attributable to the trust created under the deed dated November 28 1941 In respect of the assessment for subsequent years viz 1962 63 to 1973 74 the Tribunal referred only one question which was as follows Whether on the facts and in the circumstances of the case the income of the assessee was entitled to exemption under section 11 of the Income tax Act 1961 read with section 13 and was number therefore assessable to tax for the assessment years 1962 63 to 1973 74"
predicted_label = predict(text_input)
print(f"Predicted Label: {predicted_label}")

Predicted Label: In favor of the appellant


In [21]:

model_load_path = '/content/gdrive/MyDrive/saved_model'
tokenizer = AutoTokenizer.from_pretrained(model_load_path)
model = AutoModelForSequenceClassification.from_pretrained(model_load_path)

def predict(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

    # Make predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted label
    logits = outputs.logits
    predicted_class_id = torch.argmax(logits, dim=-1).item()

    # Map the predicted label to the corresponding class name
    label_map = {0: 'Favor of the appellant', 1: 'Against the appellant'}  # Adjust this mapping as per your dataset
    predicted_label = label_map[predicted_class_id]

    return predicted_label

# Example usage
text_input = "the appeal is directed against the judgment and order of conviction and sentence dated 27072015passed by the learned Sessions Judge Lohardaga in Sessions Trial No 413 1Arti Aarti Sukar Kumari 2Jama Sushil Munda 3Birsa Munda Runka Munda 4Boda Mundo Suman Munda Appellants The State of Jharkhand For the Respondent Mr Birendra Kumar Advocate Mr Vijay Kumar Sinha PP Present HON BLE MR JUSTICE RAJESH KUMAR and 201 34 of IPC and have been sentenced to undergo SI for life and a fine of Rs 10 000 for the offences punishable under Section 302 34 of 2013 he came to know that the son of the 2 informant had gone to his in laws place on 06022013 he was not getting well . the appellants have been convicted for the offense under Section 201, 34 of the IPC . he and his another son reached Panchpadwa in the district of . alleged that the informant and his family were thinking of taking Mansukh Munda to Devas for treatment and this intention was disclosed to his brother Hadu Munda and on this issue there appeared a fissure in the relationship between the families . the family of the informants had sent his son to bring them back but they did not return . there was friction between both sides for practicing witchcraft which resulted in the occurrence The informant could come to know that his son Markas Munda had gone to the house of Bauda Munda was dragged towards the jungle and assaulted him with fists slaps and danda by 4 5 persons which led to his death . it has been claimed that all the accused persons had left for Panchpadwa and his brother had sent him to bring him back but he had been treated at various places he was hit by a big stone leading to the death of his son in law and he has seen the incident and there were friction between the sides for practice of witchcraft . as Munda being assaulted with lathi and danda by 4 5 persons he has further stated that after assaulting him he was dragged towards the forest where he had been hit by a big stone leading to his death . a court question this witness has stated that it was dark when the incident had taken place He has further deposed that due to darkness persons who were standing at a distance could not be recognized He has also stated that he does not know when his son in law had come to his house after coming from Kisko block he and PW 1 were attacked by the accused persons and then he is in the house and on the next date the father of Markas Munda was hit by the big stone . as a result of the attack he went to his home and that on the date of occurrence 3 and Aarti Kumari Sukar Kumari, . and . the villagers had heard the sound of . the cause of death according to this witness was due to Haemorrhage and shock caused by hard and blunt substance This witness has proved the post mortem report which has been marked as Exhibit 1 PW 5 Dedo Munda is the informant who has stated that on the date of occurrence he was at his home and his son Markas Munda had gone to his in laws place at village Panchpadwa . on next date Sohrai Munda informed on phone that his elder brother has been done to death by the accused persons were ante morten in nature and time elapsed from death 6 to 36 hours . the reason for the occurrence is that he has also stated that his son was assaulted and did to death . he had been in the house and on the same date he went to the home of his brother sohrai had disclosed to him that his brother was sick PW 3 Ram Nath Munda was sick and that on phone, his son had been a witness . he has stated that his son had gone to call the brother of the informant namely Hadu Munda . the reason for the occurrence according to this witness was that the 11 PW 7 Supa Kongari has deposed that on 06022013 he was posted as Sub Inspector on Jobang Police Station He has recorded the Fard Beyan of Jairam Munda which has been proved and marked as Exhibit 2 The formal FIR is in his hand writing and signature which has be marked as exhibit 3 The occurrence is at village Panchpadwa Tongri which is an isolated place surrounded by jungle and bushes and he also stated that he and his brother were on friendly terms with him and they were on a distance of half km from the place of occurrence he had obtained the post mortem report and on direction of his superior officials he said that the re statement of Baura Munda Jama Munda and . counsel submits that the conduct of PW 1 and PW 2 had stated that it was a dark night and there is no electricity in the village and as such it would have been absolutely impossible to have identified the assailants Learned counsel further while referring to the evidence . the IO has disclosed that the distance between the place of occurrence and the house of pW 1 is about half a km . but neither had he tried to save him nor did he inform any of the villagers immediately learned PP for the State on the other hand while relying on the evidence of . that the occurrence relating to the son of the informant has stated that . if his version is taken to be true to the effect that he claims to have seen the accused persons committing the murder of his son in law but neither has he . the edifice crumbles once the cross examination of PW 1 and PW 2 is taken into consideration . he has stated that from the place of occurrence he was at a distance of 150 feet and considering the darkness which had engulfed the area and the distance from where the incident is said to have been seen by PW1 it totally nullifies the prosecution case with respect to the identification of the appellants being the assailants . however, the falsity of the allegations levelled against the villagers is a dark night According to the evidence of the so called eye witnesses he out of fear had returned back to his residence which conduct appears to be totally contrary to what one would expect in such a situation when the son in law of the Pw 1 is being done to death and that he had not seen any 7 person in the place occurrence and that it was difficult for the persons to be recognized who were standing at the distance which was about a half km and the false implication of the . the prosecution therefore having miserably failed to prove its case beyond all reasonable doubt the appellants deserves to be acquitted from the charges against them In such circumstances this appeal stands allowed and the impugned judgment and order of conviction and sentence dated 27072015 and 31072015 respectively passed by the learned Sessions Judge Lohardaga in Sessions Trial No 413 are set aside . the convicted appellant no 1 is on bail she is discharged from the liability of her bail bonds and directed to be released immediately and forthwith"
predicted_label = predict(text_input)
print(f"Predicted Label: {predicted_label}")

# HAN cross verification

Predicted Label: Against the appellant
